In [3]:
import pandas as pd

C:\Users\Deutsch\AppData\Local\Temp\ipykernel_5492\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
tdf = pd.read_csv('TM (2).csv')
tdf

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-01,118.000000,119.330002,116.500000,116.709999,116.709999,398100
1,2020-04-02,115.910004,118.120003,115.720001,117.820000,117.820000,292600
2,2020-04-03,114.809998,115.690002,113.980003,115.000000,115.000000,337100
3,2020-04-06,120.000000,122.809998,119.400002,122.410004,122.410004,328200
4,2020-04-07,125.570000,125.669998,121.900002,121.900002,121.900002,239600
...,...,...,...,...,...,...,...
980,2024-02-23,235.000000,235.759995,234.339996,235.000000,235.000000,279000
981,2024-02-26,237.589996,238.779999,236.539993,238.130005,238.130005,332900
982,2024-02-27,238.190002,238.839996,236.589996,238.039993,238.039993,301000
983,2024-02-28,237.929993,238.190002,237.000000,237.289993,237.289993,168700


In [30]:
import numpy as np

def markov_model(prices, lookback=20):

    # Calculate the percentage change in stock price
    Y = np.diff(prices) / prices[:-1]

    # Calculate the moving standard deviation
    std_dev = np.zeros(len(Y))
    for i in range(lookback, len(Y)):
        std_dev[i] = np.std(Y[i-lookback:i])

    # Define the states
    states = np.zeros(len(Y), dtype=int)
    for i in range(len(Y)):
        if Y[i] < -2*std_dev[i]:
            states[i] = 0  # B3
        elif -2*std_dev[i] <= Y[i] < -std_dev[i]:
            states[i] = 1  # B2
        elif -std_dev[i] <= Y[i] < 0:
            states[i] = 2  # B1
        elif Y[i] >= 2*std_dev[i]:
            states[i] = 3  # S3
        elif 2*std_dev[i] > Y[i] >= std_dev[i]:
            states[i] = 4  # S2
        elif std_dev[i] > Y[i] >= 0:
            states[i] = 5  # S1

    # Calculate the transition probabilities
    transitions = np.zeros((6, 6))
    for i in range(len(Y)-1):
        transitions[states[i], states[i+1]] += 1
    transitions = transitions / transitions.sum(axis=0)
    transitions = transitions / transitions.sum(axis=1)[:, np.newaxis]

    return states, transitions


states, transitions = markov_model(tdf['Adj Close'])

print("States:")
print(states)
print("\nTransitions:")
print(transitions)

States:
[3 0 3 0 3 0 0 3 0 0 3 0 0 3 3 3 3 3 3 0 2 2 5 2 2 5 5 1 1 2 2 4 2 5 1 2 3
 4 5 2 5 5 5 2 4 5 2 5 0 4 2 5 2 5 2 5 5 2 2 2 5 2 2 4 5 1 5 2 5 5 4 5 2 5
 5 2 2 2 2 3 1 1 1 0 4 4 4 3 5 5 4 4 2 2 5 5 2 2 2 5 5 5 2 5 2 2 5 1 3 1 2
 2 5 4 5 5 5 2 0 5 5 2 5 4 1 1 2 2 5 2 5 5 2 2 5 2 2 2 5 5 4 5 2 2 2 0 4 2
 3 3 1 3 5 4 5 4 1 4 4 2 2 2 5 5 5 1 2 0 5 5 5 5 2 5 4 4 3 5 2 5 2 2 2 2 5
 2 5 5 5 5 2 5 5 2 5 5 2 2 5 0 2 4 4 0 2 2 0 5 0 2 4 3 2 4 5 5 3 2 5 1 5 1
 5 2 5 2 1 2 5 2 2 2 5 2 4 5 2 5 4 2 5 3 5 0 1 2 5 4 5 2 4 2 5 2 5 2 5 2 5
 5 5 5 2 1 5 2 5 1 2 5 2 1 4 1 5 4 5 4 0 4 4 5 3 4 2 5 5 4 2 4 5 5 3 3 4 4
 2 2 5 2 5 2 5 5 5 0 5 5 2 2 5 2 2 2 5 5 2 5 2 4 5 5 5 2 2 2 4 4 1 4 2 2 5
 4 1 4 3 0 5 1 2 2 5 2 5 2 1 1 0 1 4 5 4 2 5 2 5 5 5 4 5 2 5 1 5 5 5 5 2 1
 3 2 5 2 4 2 5 0 2 0 5 1 5 5 4 5 2 2 5 5 5 2 1 2 2 5 5 5 5 4 2 5 5 1 5 0 5
 5 3 2 4 2 2 5 2 2 2 0 1 5 5 3 2 5 4 1 2 5 0 4 3 5 1 2 5 2 5 5 2 2 5 5 5 3
 2 5 5 2 5 4 4 2 2 1 2 2 2 2 2 1 5 4 2 5 2 2 5 4 5 0 2 1 5 5 1 5 1 2 2 4 2
 2 1 1 0 0 1 3 5 

In [34]:
def markov_model2(prices, lookback=20):

    # Calculate the percentage change in stock price
    Y = np.diff(prices) / prices[:-1]

    # Calculate the moving standard deviation
    std_dev = np.zeros(len(Y))
    for i in range(lookback, len(Y)):
        std_dev[i] = np.std(Y[i-lookback:i])

    # Define the states with labels
    states = np.empty(len(Y), dtype=object)
    for i in range(len(Y)):
        if Y[i] < -2*std_dev[i]:
            states[i] = "B3"
        elif -2*std_dev[i] <= Y[i] < -std_dev[i]:
            states[i] = "B2"
        elif -std_dev[i] <= Y[i] < 0:
            states[i] = "B1"
        elif Y[i] >= 2*std_dev[i]:
            states[i] = "S3"
        elif 2*std_dev[i] > Y[i] >= std_dev[i]:
            states[i] = "S2"
        elif std_dev[i] > Y[i] >= 0:
            states[i] = "S1"

    # Calculate the transition probabilities
    transitions = np.zeros((6, 6))
    states_mapping = {"B3": 0, "B2": 1, "B1": 2, "S3": 3, "S2": 4, "S1": 5}
    for i in range(len(Y)-1):
        transitions[states_mapping[states[i]], states_mapping[states[i+1]]] += 1
    transitions = transitions / transitions.sum(axis=0)
    transitions = transitions / transitions.sum(axis=1)[:, np.newaxis]

    return states, transitions


states, transitions = markov_model2(tdf['Adj Close'])

print("States:")
print(states)
print("\nTransitions:")
print(transitions)

States:
['S3' 'B3' 'S3' 'B3' 'S3' 'B3' 'B3' 'S3' 'B3' 'B3' 'S3' 'B3' 'B3' 'S3'
 'S3' 'S3' 'S3' 'S3' 'S3' 'B3' 'B1' 'B1' 'S1' 'B1' 'B1' 'S1' 'S1' 'B2'
 'B2' 'B1' 'B1' 'S2' 'B1' 'S1' 'B2' 'B1' 'S3' 'S2' 'S1' 'B1' 'S1' 'S1'
 'S1' 'B1' 'S2' 'S1' 'B1' 'S1' 'B3' 'S2' 'B1' 'S1' 'B1' 'S1' 'B1' 'S1'
 'S1' 'B1' 'B1' 'B1' 'S1' 'B1' 'B1' 'S2' 'S1' 'B2' 'S1' 'B1' 'S1' 'S1'
 'S2' 'S1' 'B1' 'S1' 'S1' 'B1' 'B1' 'B1' 'B1' 'S3' 'B2' 'B2' 'B2' 'B3'
 'S2' 'S2' 'S2' 'S3' 'S1' 'S1' 'S2' 'S2' 'B1' 'B1' 'S1' 'S1' 'B1' 'B1'
 'B1' 'S1' 'S1' 'S1' 'B1' 'S1' 'B1' 'B1' 'S1' 'B2' 'S3' 'B2' 'B1' 'B1'
 'S1' 'S2' 'S1' 'S1' 'S1' 'B1' 'B3' 'S1' 'S1' 'B1' 'S1' 'S2' 'B2' 'B2'
 'B1' 'B1' 'S1' 'B1' 'S1' 'S1' 'B1' 'B1' 'S1' 'B1' 'B1' 'B1' 'S1' 'S1'
 'S2' 'S1' 'B1' 'B1' 'B1' 'B3' 'S2' 'B1' 'S3' 'S3' 'B2' 'S3' 'S1' 'S2'
 'S1' 'S2' 'B2' 'S2' 'S2' 'B1' 'B1' 'B1' 'S1' 'S1' 'S1' 'B2' 'B1' 'B3'
 'S1' 'S1' 'S1' 'S1' 'B1' 'S1' 'S2' 'S2' 'S3' 'S1' 'B1' 'S1' 'B1' 'B1'
 'B1' 'B1' 'S1' 'B1' 'S1' 'S1' 'S1' 'S1' 'B1' 'S1' 'S1' 'B1' 'S1' 'S1